In [1]:
import intera_interface
import rospy
import json

In [2]:
rospy.init_node("sawyer_example")

In [3]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [4]:
rs.enable()

[INFO] [1554560557.122703]: Robot Enabled


In [5]:
limb = intera_interface.Limb('right')

In [6]:
angles = limb.joint_angles()

In [7]:
angles

{'right_j0': 0.00175390625,
 'right_j1': -1.177833984375,
 'right_j2': 0.0013876953125,
 'right_j3': 2.17882421875,
 'right_j4': 0.0031181640625,
 'right_j5': 0.56447265625,
 'right_j6': 3.3151767578125}

In [8]:
limb.move_to_neutral()

In [10]:
head = intera_interface.head.Head()

In [12]:
head.pan()

-3.1976141929626465

In [15]:
head.set_pan(-1.57)

In [22]:
hope_lower_left = limb.joint_angles()

In [17]:
hope_lower_right

{'right_j0': 0.0704453125,
 'right_j1': -3.5299384765625,
 'right_j2': -3.023064453125,
 'right_j3': -2.736953125,
 'right_j4': -1.4090927734375,
 'right_j5': 1.5120576171875,
 'right_j6': 1.0290107421875}

In [19]:
hope_lower_right = limb.joint_angles()

In [40]:
hope_higher_right = limb.joint_angles()

In [30]:
hope_lower_left = limb.joint_angles()

## Read poses

In [9]:
with open("../config/poses.json") as f:
    extremes = json.load(f)

In [13]:
hope_extremes = [extremes['hope']['higher']['left'], extremes['hope']['higher']['right'],
                 extremes['hope']['lower']['left'], extremes['hope']['lower']['right']]
for i in range(1):
    for pose in hope_extremes:
        limb.move_to_joint_positions(pose)
        rospy.sleep(2)

## Overwrite poses

In [65]:
hope = {"higher": {"left" : hope_higher_left, "right": hope_higher_right},
        "lower": {"left" : hope_lower_left, "right": hope_lower_right}}

In [66]:
fear = {}

In [67]:
extremes = {"hope": hope, "fear": fear}

In [68]:
with open("../config/poses.json", "w") as f:
    json.dump(extremes, f, indent=4)

# Kinematics

In [9]:
from sawyer.transformations import list_to_pose

In [10]:
from geometry_msgs.msg import Pose

In [16]:
limb.ik_request(list_to_pose([[0.452, 0.159, 0.216],  [0.707, 0.707, 0.002, 0.004]]))

[ERROR] [1554312105.446198]: INVALID POSE - No Valid Joint Solution Found.


False

In [15]:
limb.joint_angles()

{'right_j0': 0.0018564453125,
 'right_j1': -1.1825615234375,
 'right_j2': -0.0019326171875,
 'right_j3': 2.177814453125,
 'right_j4': 0.0020849609375,
 'right_j5': 0.5650791015625,
 'right_j6': 3.3153837890625}